In [ ]:
%cd /content/drive/MyDrive/
# %mkdir cc
%cd img_align_celeba/img_align_celeba
%pwd



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch.nn as nn
import torch

In [ ]:
from __future__ import print_function

import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [ ]:

dataroot = "/content/drive/MyDrive/cc/img_align_celeba"


workers = 2


batch_size = 128


image_size = 64


nc = 3


nz = 100


ngf = 64


ndf = 64

num_epochs = 15


lr = 0.0002


beta1 = 0.5


ngpu = 1

In [ ]:

dataset = dset.ImageFolder(root=dataroot,
                           transform=transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                           ]))

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)


device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


real_batch = next(iter(dataloader))
plt.figure(figsize=(8,8))
plt.axis("off")
plt.title("Training Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=2, normalize=True).cpu(),(1,2,0)))

In [ ]:
print(device)

In [ ]:

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:


class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.nz = nz
        self.ngf = ngf
        self.nc = nc

        self.main = nn.Sequential(
            
            nn.Linear(nz, ngf  * 4 * 4),
            nn.BatchNorm1d(ngf  * 4 * 4),
            nn.ReLU(True),
            
            nn.Linear(ngf  * 4 * 4, ngf  * 8 * 8),
            nn.BatchNorm1d(ngf  * 8 * 8),
            nn.ReLU(True),
            
            nn.Linear(ngf  * 8 * 8, nc * 64 * 64),
            nn.Tanh()
           
        )

    def forward(self, input):
        
        input = input.view(input.size(0), self.nz)
        output = self.main(input)
        return output.view(input.size(0), self.nc, 64, 64)

In [ ]:

netG = Generator(ngpu).to(device)


if (device.type == 'cuda') and (ngpu > 1):
    netG = nn.DataParallel(netG, list(range(ngpu)))


netG.apply(weights_init)

print(netG)

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, ngpu):
        super(Discriminator, self).__init__()
        self.ngpu = ngpu
        self.nc = nc
        self.ndf = ndf

        self.main = nn.Sequential(
           
            nn.Linear(nc * 64 * 64, ndf * 4 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(ndf * 4 * 4, ndf * 2 * 4 * 4),
            nn.BatchNorm1d(ndf * 2 * 4 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(ndf * 2 * 4 * 4, ndf * 4 * 4 * 4),
            nn.BatchNorm1d(ndf * 4 * 4 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(ndf * 4 * 4 * 4, ndf * 8 * 4 * 4),
            nn.BatchNorm1d(ndf * 8 * 4 * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Linear(ndf * 8 * 4 * 4, 1),
            nn.Sigmoid()
        )

    def forward(self, input):
        input = input.view(input.size(0), -1)
        return self.main(input)


In [ ]:

netD = Discriminator(ngpu).to(device)


if (device.type == 'cuda') and (ngpu > 1):
    netD = nn.DataParallel(netD, list(range(ngpu)))


netD.apply(weights_init)
print(netD)

In [ ]:

criterion = nn.BCELoss()


fixed_noise = torch.randn(64, nz, 1, 1, device=device)


real_label = 1.
fake_label = 0.


optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [ ]:

img_list = []
G_losses = []
D_losses = []
ff=[]
iters = 0

print("Starting Training Loop...")

for epoch in range(num_epochs):
  
    for i, data in enumerate(dataloader, 0):

        
        netD.zero_grad()
        
        real_cpu = data[0].to(device)
        b_size = real_cpu.size(0)
        label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
     
        output = netD(real_cpu).view(-1)
       
        errD_real = criterion(output, label)
       
        errD_real.backward()
        D_x = output.mean().item()

        
        noise = torch.randn(b_size, nz, 1, 1, device=device)
     
        fake = netG(noise)
        label.fill_(fake_label)
   
        output = netD(fake.detach()).view(-1)
        
        errD_fake = criterion(output, label)
        
        errD_fake.backward()
        D_G_z1 = output.mean().item()

        errD = errD_real + errD_fake
        
        optimizerD.step()

        
        netG.zero_grad()
        label.fill_(real_label)  
  
        output = netD(fake).view(-1)
        
        errG = criterion(output, label)
        
        errG.backward()
        D_G_z2 = output.mean().item()
        
        optimizerG.step()

        
        if i % 50 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(dataloader),
                     errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))

        
        G_losses.append(errG.item())
        D_losses.append(errD.item())

        
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
                ff.append(fake)
            img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D")
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.axis("off")
ims = [[plt.imshow(np.transpose(i,(1,2,0)), animated=True)] for i in img_list]
ani = animation.ArtistAnimation(fig, ims, interval=1000, repeat_delay=1000, blit=True)

HTML(ani.to_jshtml())

In [ ]:

real_batch = next(iter(dataloader))


plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))


plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(img_list[-1],(1,2,0)))
plt.show()

In [ ]:
import os
from PIL import Image
import numpy as np

save_folder_path = '/content/drive/MyDrive/c/'


os.makedirs(save_folder_path, exist_ok=True)


for index, img in enumerate(ff):
    file_name = f'image_{index}.png'  
    file_path = os.path.join(save_folder_path, file_name)
    

    np_img = img.detach().cpu().numpy()  
    np_img = np.transpose(np_img, (1, 2, 0))  
    Image.fromarray((np_img * 255).astype(np.uint8)).save(file_path)  


In [ ]:
import os
from PIL import Image
import numpy as np

save_folder_path = '/content/drive/MyDrive/c/'


os.makedirs(save_folder_path, exist_ok=True)


for batch_index, img_batch in enumerate(ff):
    for img_index, img in enumerate(img_batch):
        file_name = f'image_{batch_index}_{img_index}.png'  
        file_path = os.path.join(save_folder_path, file_name)

  
        np_img = img.detach().cpu().numpy()  
        np_img = np.transpose(np_img, (1, 2, 0)) 
        np_img = (np_img * 255).astype(np.uint8)  
        Image.fromarray(np_img).save(file_path)
